In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import date
import datetime
import csv
import time
import os
from operator import itemgetter
import pandas as pd
import xlsxwriter

In [2]:
"""
1. you must have a stock list xlsx file in the folder with stocks that you want to pull data
2. you must have a folder called Stocks
"""

'\n1. you must have a stock list xlsx file in the folder with stocks that you want to pull data\n2. you must have a folder called Stocks\n'

In [3]:
today = str (date.today())
print(today)
# reading stock list
list_loc = "Stocks.xlsx"
#number of time that you want to repeat
n_repeat = 3
#location of the reports
path_3 = r"Reports\%sReport.xlsx" % today
# need to fix when more than one date is missing


2019-02-19


In [4]:
def timer(counter):
    if counter % 30 == 0 or counter % 25 == 0:
        print('waiting')
        time.sleep(0.2)

def df_add(df, seq, i, last_date, last_closed):
    #adding data into column
    new_df = [seq, i, last_date, last_closed] 
    print(new_df)
    df.loc[-1] = new_df
    #fixing index
    df.index = df.index + 1

def df_sort(df):
    df = df.sort_values(by = ['SEQ'], ascending = False, axis = 0)
    return df

    
def mine_data(i, finder):
    url = "https://finance.yahoo.com/quote/"+i+"/history"
    headers = {'User-Agent': 'Mozilla/5.0'}
    page = requests.get(url,headers=headers)
    bsobj = BeautifulSoup(page.content, "html.parser")
    data = bsobj.find("span",{"data-reactid":"%s" %finder})
    if data == None:
        data = None
    else:
        data = ' '.join([i.strip() for i in data])
        data = data.replace(",", "")
    return data

def int_date(date):
    date = str(date.replace(',',''))
    return int(datetime.datetime.strptime(date, "%b %d %Y").strftime('%Y%m%d'))

In [5]:
class finder:
    def __init__(self, loc):
        self.loc = loc
        self.name = self
        
    def add(self, x):
        self.loc += x
        
    #next day for normal stocks
    def n_s_n(self):
        x = 15
        self.add(x)
        
    #next day for normal funds
    def n_f_n(self):
        y = 14
        self.add(y)
    
    #next day for stocks with dividend 
    def d_s_n(self):
        z = 22
        self.add(z)
        
    #next day for funds with dividend 
    def d_f_n(self):
        k = 21
        self.add(k)
    
    def empty_date(self):
        x = -1
        self.add(x)

In [6]:
def new_data(n_repeat, i, path_2):
    #creating a new file and dataframe
    print('New Stock: %s'%i)
    df_n = pd.DataFrame(columns = ['SEQ', 'Stock Name', 'Date', 'Closed Price'])
    
    #setting
    last_date_finder = finder(51)
    last_closed_finder = finder(61)
    second_date_finder = finder(66)

    n_repeat = n_repeat
    counter = 0
    fund_finder = False
    
    #Data minning
    while n_repeat > 0:
        counter += 1
        if counter %30 == 0 or counter %25 ==0:
            print('waiting')
            timer(counter)
            print(counter)
        else:
            print(counter)   
        
        last_date = mine_data(i, last_date_finder.loc)
        seq = int_date(last_date)
        
        second_date = mine_data(i, second_date_finder.loc)
        if second_date == None:
            print('Second date empty')
            time.sleep(0.01)
            print('Funds')
            # moving it to next date
            second_date_finder.empty_date()
            fund_finder = True
            second_date = mine_data(i, second_date_finder.loc)
        seq_s = int_date(second_date)  
        
        #normal setting
        if seq != seq_s:
            last_closed = mine_data(i, last_closed_finder.loc)
            df_add(df_n, seq, i, last_date, last_closed)
            time.sleep(0.02)
            n_repeat -= 1
            if fund_finder == True:
                last_date_finder.n_f_n()
                last_closed_finder.n_f_n()
                second_date_finder.n_f_n()
            else:
                last_date_finder.n_s_n()
                last_closed_finder.n_s_n()
                second_date_finder.n_s_n()
                
        elif seq == seq_s:
            print('Dividend')
            last_closed = mine_data(i, last_closed_finder.loc)
            df_add(df_n, seq, i, last_date, last_closed)
            n_repeat -= 1
            
            if fund_finder == True:
                last_date_finder.d_f_n()
                last_closed_finder.d_f_n()
                second_date_finder.n_f_n()
            else:
                last_date_finder.d_s_n()
                last_closed_finder.d_s_n()
                second_date_finder.d_s_n()
        else:
            print('Error')      
    df_n.to_csv(path_2) 

In [7]:
def old_data(n_repeat, i, path_2):
    #reading an exisiting file
    print('Updating existing stock: %s'%i)
    df = pd.read_csv(path_2)
    df = df[['SEQ', 'Stock Name', 'Date', 'Closed Price']]
    
    #setting
    last_date_finder = finder(51)
    last_closed_finder = finder(61)
    second_date_finder = finder(66)
    counter = 0
    fund_finder = False
    
    seq_o = df['SEQ'].max()
    
    last_date = mine_data(i, last_date_finder.loc)
    seq = int_date(last_date)
    second_date = mine_data(i, second_date_finder.loc)
    if second_date == None:
        seq_s = int_date(mine_data(i, second_date_finder.loc - 1))
    else:
        seq_s = int_date(second_date)
    
    if seq-seq_o > 1:
        print("data missing")
        os.remove(path_2)
        time.sleep(1)
        print('csv file deleted')
        new_data(n_repeat, i, path_2)
        
    else:
        while seq_o != seq:
            last_date = mine_data(i, last_date_finder.loc)
            seq = int_date(last_date)
            second_date = mine_data(i, second_date_finder.loc)

            #stocks with dividend
            if second_date == None:
                print('second date empty')
                time.sleep(0.2)
                second_date = mine_data(i, second_date_finder.loc)
                print('funds')
                second_date_finder.empty_date()
                fund_finder = True
                second_date = mine_data(i, second_date_finder.loc)

            first_closed_finder = finder(61)
            first_closed = mine_data(i, first_closed_finder.loc)

            if first_closed == None:
                print('Dividend for wmt')
                time.sleep(0.2)
                last_date_finder = finder(58)
                last_closed_finder = finder(66)
                second_date_finder = finder(73)
                second_date = mine_data(i, second_date_finder.loc)
                last_closed = mine_data(i, last_date_finder.loc)

            seq_s = int_date(second_date)

            #normal stocks
            if seq != seq_s:
                last_closed = mine_data(i, last_closed_finder.loc)
                if first_closed != None:
                    df_add(df, seq, i, last_date, last_closed)
                    df = df_sort(df)
                    time.sleep(0.0123)
                    if fund_finder == True:
                        last_date_finder.n_f_n()
                        last_closed_finder.n_f_n()
                        second_date_finder.n_f_n()
                    else:
                        last_date_finder.n_s_n()
                        last_closed_finder.n_s_n()
                        second_date_finder.n_s_n()

                elif seq == seq_s:
                    print('Dividend')
                    last_closed = mine_Data(i, last_closed_finder.loc)
                    df_add(df, seq, i, last_date, last_closed)
                    df_sort(df)
                    last_date_finder.d_s_n()
                    last_closed_finder.d_s_n()
                    second_date_finder.d_s_n()
                else:
                    print('error')
                seq_o = seq
                print(seq_o)
            else:
                print('end')
        df.to_csv(path_2)

In [8]:
def analyze_data(df_f, stock_list, columns):
    for i in stock_list:
        path_2 = "Stocks\%s.csv" %i
        df_o = pd.read_csv(path_2)
        df_f = df_f
        current_seq = df_o.loc[0,"SEQ"]
        current_price = df_o.loc[0,"Closed Price"]
        # creating averages
        days = list(range(10,101,10))
        aver ={}
        for day in days:
            aver[day] = 1-(df_o.loc[0:(day-1),"Closed Price"].mean()/current_price)
        df_f.loc[len(df_f)]=[current_seq,stock_dic_type[i],i,stock_dic[i], 
                             note_dic[i], current_price, buy_dic[i], 
                             (current_price - buy_dic[i])/ buy_dic[i],
                             aver[10],aver[20],aver[30],aver[40],aver[50],aver[60],aver[70],aver[80],aver[90],aver[100]]
    return df_f

In [9]:
def organize_report(df_f, path_3, stock_list):
    #formating data
    df_f = df_f.sort_values(by=['Stock_Type', '10A'])
    print(df_f)
    path_3 = path_3
    writer = pd.ExcelWriter(path_3, engine='xlsxwriter')
    df_f.to_excel(writer, sheet_name='Sheet1')
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    format_data = workbook.add_format({'num_format': '0.00%'})
    worksheet.set_column('I2:Z99', None, format_data)

    format1 = workbook.add_format({'bg_color':   '#FFC7CE','font_color': '#9C0006'})
    worksheet.conditional_format('I2:Z99', {'type': 'cell', 'criteria': '<', 'value': 0, 'format': format1})

    worksheet.set_column('E:E', 23)

    format2 = workbook.add_format({'bg_color':   '#FFFF00'})
    worksheet.conditional_format('C1:C999', {'type': 'text', 'criteria': 'containing', 'value' : 'Index', 'format': format2})

    format3 = workbook.add_format({'bg_color':   '#008000'})
    worksheet.conditional_format('C1:C999', {'type': 'text', 'criteria': 'containing', 'value' : 'Owned', 'format': format3})

    writer.save()

    #open a file

    os.startfile(path_3)

In [10]:
#setting
df1 = pd.read_excel(list_loc)
stock_list = df1["Stocks"]
stock_names = df1["Stock_Name"]
stock_dic = dict(zip(stock_list, stock_names))
stock_type = df1["Stock_Type"]
stock_dic_type = dict(zip(stock_list,stock_type))
buy_price = df1["Buy_Price"]
buy_dic = dict(zip(stock_list, buy_price))
note = df1["Note"]
note_dic = dict(zip(stock_list, note))

In [12]:
# excuting functions
for i in stock_list:
    # if it is a new stock
    #location of stock files
    path_2 = "Stocks\%s.csv" %i
    if os.path.isfile(path_2) == False:
        new_data(n_repeat, i, path_2)
    else:
        old_data(n_repeat, i, path_2)

print('Analyzing all stock data')
columns = ['Date', 'Stock_Type', 'Stock_Symbol', 'Stock_Name', 'Note', 'Current Price', 'Buy Price', 'Gain / Loss',  '10A', '20A', '30A','40A','50A','60A','70A','80A','90A','100A']        
df_f = pd.DataFrame(columns = columns)
df_f = analyze_data(df_f, stock_list, columns)
    
organize_report(df_f, path_3, stock_list)


Updating existing stock: BRK-B
Analyzing all stock data
        Date      Stock_Type Stock_Symbol  \
3   20190215           Index           GE   
1   20190215           Index        ^IXIC   
0   20190215           Index         ^DJI   
2   20190215           Index        ^GSPC   
14  20190215           Owned        COMVX   
8   20190215           Owned        FHKCX   
4   20190215           Owned        BEARX   
12  20190215           Owned        LETRX   
13  20190215           Owned        FSNVX   
9   20190215           Owned        FSCHX   
5   20190215           Owned        FSAGX   
6   20190215           Owned        FTQGX   
10  20190215           Owned        FSCOX   
7   20190215           Owned          WMT   
15  20190215            Sold        FEMKX   
16  20190215            Sold          MCD   
17  20190215  Sold with loss         INTU   
19  20190215       Watchlist         AMZN   
18  20190215       Watchlist        FSUTX   
20  20190215       Watchlist        BRK-B   